# Eager Execution で遊ぶ
いろいろ試してみる

In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

# 単純計算

In [3]:
x = tf.constant(1.)
y = tf.constant(2.)
x

<tf.Tensor: id=1, shape=(), dtype=float32, numpy=1.0>

In [4]:
for i in range(10):
    z = y
    y = x + y
    x = z
    print(y)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(13.0, shape=(), dtype=float32)
tf.Tensor(21.0, shape=(), dtype=float32)
tf.Tensor(34.0, shape=(), dtype=float32)
tf.Tensor(55.0, shape=(), dtype=float32)
tf.Tensor(89.0, shape=(), dtype=float32)
tf.Tensor(144.0, shape=(), dtype=float32)
tf.Tensor(233.0, shape=(), dtype=float32)


# パフォーマンス

In [5]:
import time
from tqdm import tqdm

In [6]:
def calc():
    X = tf.ones(shape=[256, 256], dtype=tf.float32)
    for i in range(100):
        X = tf.layers.dense(X, 256)
    return X

In [7]:
start = time.time()
with tf.Graph().as_default():
    X = calc()
    after_graph = time.time()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        result = sess.run(X)
end = time.time()
print('graph mode elapsed:', end - start)
print('graph mode run:', end - after_graph)
#print(result)

graph mode elapsed: 1.3104174137115479
graph mode run: 0.4512958526611328


In [8]:
start = time.time()
result = calc()
print('eager mode elapsed:', time.time() - start)
#print(result)

eager mode elapsed: 0.25153040885925293


In [9]:
start = time.time()
elapsed_list = []
with tf.Graph().as_default():
    X = calc()
    with tf.Session() as sess:
        for i in tqdm(range(50)):
            iter_start = time.time()
            sess.run(tf.global_variables_initializer())
            result = sess.run(X)
            elapsed_list.append(time.time() - iter_start)

print('graph mode elapsed:', time.time() - start)
print(' '.join([f'{elapsed:.3f}' for elapsed in elapsed_list]))
#print(result)

100%|██████████| 50/50 [00:12<00:00,  4.00it/s]

graph mode elapsed: 13.499590873718262
0.340 0.236 0.237 0.238 0.240 0.237 0.239 0.236 0.236 0.236 0.244 0.239 0.247 0.245 0.248 0.249 0.246 0.246 0.249 0.250 0.249 0.251 0.244 0.245 0.248 0.250 0.248 0.247 0.252 0.243 0.249 0.249 0.250 0.252 0.254 0.249 0.250 0.251 0.258 0.258 0.255 0.256 0.255 0.255 0.262 0.256 0.256 0.252 0.252 0.256


In [10]:
start = time.time()
elapsed_list = []

for i in tqdm(range(50)):
    iter_start = time.time()
    result = calc()
    elapsed_list.append(time.time() - iter_start)
print('eager mode elapsed:', time.time() - start)
print(' '.join([f'{elapsed:.3f}' for elapsed in elapsed_list]))
#print(result)

100%|██████████| 50/50 [00:21<00:00,  2.35it/s]

eager mode elapsed: 21.26130485534668
0.255 0.260 0.264 0.268 0.277 0.280 0.289 0.360 0.302 0.308 0.316 0.320 0.335 0.342 0.350 0.356 0.362 0.371 0.377 0.385 0.392 0.407 0.411 0.415 0.427 0.429 0.434 0.440 0.447 0.451 0.460 0.468 0.474 0.482 0.488 0.498 0.504 0.511 0.518 0.525 0.541 0.549 0.537 0.557 0.562 0.570 0.579 0.587 0.592 0.608
